Parámetros a encontrar:
$D_A(z_i)$, $H(z_i)$, $f\sigma_8(z_i)$, $b\sigma_8$ , $P_s(z_i)$, $\omega_{b,0}$, $h$, $\omega_{m,0}$, $n_s$, $\sigma_v (z_{mean})$ y $\sigma_p (z_{mean})$


In [1]:
# Se definen los parámetros de referencia
#H = 1
#fs8 = 1
#bs8 = 1
#Ver valores reales

Om_b0 = 0.05
w_b0 = 0.022445
Om_m0 = 0.32
w_m0= 0.143648
h= 0.67
n_s= 0.96
s8= 0.816

gamma = 0.55
w_0 = -1
w_a = 0
Om_DE = 0.68

A_surv = 15000
s_z = 0.0001 #Spectroscopic redshift error

In [2]:
#P_dd = np.loadtxt("P_dd.txt")

def f(z):
  return Om_m(z)**gamma

'''
Redshift uncertainty and shot noise
'''
#def P_zerrlin(k, u, z):
#  return P_zslin(k, u, z) * F_z(k, u, z) + P_s(z)

def s_r(z):
  return (c/H(z)) * s_z

def F_z(k,u,z):
  return np.exp(-k**2 * u**2 * s_r(z)**2)

'''
AP projection effects
'''
## Cuál es D_Aref?
def qper(z):
  return D_A(z)/D_Aref(z)

def qpar(z):
  return H_ref(z)/H(z)

# To convert the known reference k_ref and mu_ref to the unknown, true k and mu
def k(k_ref, u_ref):
  return (k_ref /qper)* (1 + u_ref**2 * ((qper**2/qpar**2) -1))**(1/2)

def u(u_ref):
  return u_ref * (qper/qpar) * (1 + u_ref**2 * ((qper**2/qpar**2) -1))**(-1/2)

#def P_projlin(k_ref, u_ref,z):
#  return (1/(qper**2* qpar)) * P_zslin(k(k_ref, u_ref), u(u_ref), z)

#def P_zs(k,u,z):
#  return (1/(1+ (f(z)*k*u* sp(z))**2))* ((b(z)*s8(z)+ f(z)*s8(z)*u**2)**2) * P_dw(k,u,z)/ (s8(z)**2)

# Non-linear damping factor of the BAO signal.
def g_u(k, u, z):
  return sv(z)**2 *(1- u**2 + (u**2)*(1+ f(z))**2)

def P_dw(k, u, z):
  return P_dd(k,z)* np.exp(-g_u(k,u,z)*k**2) + P_nw(k,u,z)* (1- np.exp(-g_u(k,u,z)*k**2))

# In practice:
# sv(z) controls the strength of the non-linear damping of the BAO signal in all directions.
def sv(z):
  return sv(z_mean) * D(z) / D(z_mean)

def sp(z):
  return sp(z_mean) * D(z) / D(z_mean)

#deltaz = 0.2
#def n(z):
#  return dN(z)/(dOm* dz) * (A_surv/V_s(z))* deltaz

'''
los zi es el redshift en el centro del intervalo?
'''
# Posición centrada de los bins
z_i = [1., 1.2, 1.4, 1.6]

n_zmean = [6.86 * 10**(-4), 5.58 * 10**(-4), 4.21* 10**(-4), 2.61 * 10**(-4)]
V_s_zmean = [7.94, 9.15, 10.05, 16.22]
b_zmean = [1.46, 1.61, 1.75, 1.9]

def V_eff(zi, k, u):
  bin = z_i.index(zi)
  return V_s_zmean[bin] * (n_zmean[bin])*P_obs(k,u, zi)/(n_zmean*P_obs(k,u, zi) + 1)**2

In [3]:
# Full linear model
def P_obs_linear(k, u, z, bs8=None, fs8=None, s8= None):
  bs8_value = bs8(z) if bs8 is None else bs8
  fs8_value = fs8(z) if fs8 is None else fs8
  s8_value = s8(z) if s8 is None else s8

  return 1/(qpar**2 * qper) * (bs8_value + fs8_value*u**2)**2 * (P_dd(k,z)/s8_value**2) * F_z(k, u,z) + P_s(z)

# Full non-linear model
def P_obs(k, u, z, bs8=None, fs8=None, f= None , sp=None, s8= None):
  bs8_value = bs8(z) if bs8 is None else bs8
  fs8_value = fs8(z) if fs8 is None else fs8
  f_value = f(z) if f is None else f
  sp_value = sp(z) if sp is None else sp
  s8_value = s8(z) if s8 is None else s8

  return (1/(qpar**2 * qper))*((bs8_value + fs8_value*u**2)**2/(1+(f_value*k*u*sp_value)**2))* (P_dw(k,u,z)/s8_value**2)* F_z(k,u,z) + P_s(z)

In [4]:
# epsilon varía entre los 0.0001 y 0.01
def derivada_ln_P_obs(k, u, z, a, eps =1e-4):
    #h: Tamaño del paso (opcional, por defecto 1e-5).
    '''
    Cómo se calcula si depende de k y u?

    Cómo hago si P_dw incluye a u y no se define con u? Por qué ya no se define con u?
    Cómo se define s8? Decía q viene de los input files (pág 35)?

    Cómo ingreso los parámetros de referencia si P_obs no tiene los shape parameters en él? Al igual q H y D_A

    Cómo evalúo en sv y sp?
    Para derivadas parciales, sv(z) o sv_ref? cuando se evalúa?### Para derivadas parciales, sv(z) o sv_ref? cuando se evalúa?
    '''
    if a == 'P_s':
      ln_P_obs = np.log(P_obs)
      derivada = 1/ ln_P_obs(k, u, z)

    # if are shape parameters w_b0 , h, w_m0 , n_s

    elif a == 'w_b0' or a== 'h' or a == 'w_m0' or a == 'n_s':
      # str to function
      a= globals().get(a)
      derivada = ( np.log(P_dw(k,z, a = a*(1+ eps))/s8(z)**2)- np.log(P_dw(k,z, a = a *(1-eps))/s8(z)**2) )/ (2 * eps* a)

    # if are non-linear parameters sv y sp
    #three-point stencil derivative of f with respect to x at x0
    elif a == 'sv':
      derivada = (np.log(P_dw(k, u, z, sv = sv*(1+ eps))/ s8(z)**2)- np.log(P_dw(k, u, z, sv = sv*(1 - eps))/ s8(z)**2)) / (2 * eps * np.log(P_dw(k, u, z, sv = sv)/ s8(z)**2))

    elif a == 'sp':
      derivada = (np.log(1/ (1+(f(z)*k*u*sp(z)*(1+ eps))**2))- np.log(1/ (1+(f(z)*k*u*sp(z)*(1- eps))**2))) / (2 * eps * np.log(1/ (1+(f(z)*k*u*sp(z))**2)))

    # if parameters are fs8 and bs8
    elif a == 'fs8':
      derivada = (np.log(P_obs(k,u,z, fs8 = fs8**(1 + eps))) - np.log(P_obs(k,u,z, fs8 = fs8**(1 - eps)))) / (2* eps * np.log(fs8))

    elif a == 'bs8':
      derivada = (np.log(P_obs(k,u,z, bs8 = bs8**(1 + eps))) - np.log(P_obs(k,u, z, bs8 = bs8**(1 - eps)))) / (2* eps * np.log(bs8))

    elif a == 'H':
      derivada = 8* (np.log(P_obs(k,u,z, H= H**(1+ eps)))- np.log(P_obs(k, u, z,H= H**(1- eps))))/(12* eps * np.log(H)) - (np.log(P_obs(k,u,z, H= H**(1+ 2*eps)))- np.log(P_obs(k, u, z, H= H**(1 - 2*eps))))/(12* eps * np.log(H))
    elif a == 'D_A':
      derivada = 8* (np.log(P_obs(k,u,z, D_A = D_A**(1+ eps)))- np.log(P_obs(k, u, z, D_A = D_A**(1- eps))))/(12* eps * np.log(D_A)) - (np.log(P_obs(k,u,z, D_A = D_A**(1+ 2*eps)))- np.log(P_obs(k, u, z, D_A = D_A**(1 - 2*eps))))/(12* eps * np.log(D_A))

    return derivada



In [5]:
#ejemplo de la derivada anterior
def y(z):
  return z**2

def f(z):
  y_value = y(z)
  return y_value

def derivada_f(z, h=1e-5):
    #h: Tamaño del paso (opcional, por defecto 1e-5).

    derivada = (f(z + h*z) - f(z - h*z)) / (2 * h * z)
    return derivada

derivada = derivada_f(3)
print(derivada)

6.000000000039306


In [6]:
'''
Se debe integrar numéricamente con trapecios o simpson?
Lo hice con trapecios, está bien?
'''
#romber
#cumsum

k_min = 0
k_max = 1
#ver parámetros reales

a = 'bs8'
b = 'f'

#definimos los parámetros con los cuales derivar
k = np.linspace(k_min, k_max, 100)
u = np.linspace(-1, 1, 100)
def ec1(k, u, zi, a, b):
  return k**2 * derivada_ln_P_obs(k, u, zi, a) *derivada_ln_P_obs(k, u, zi, b)* V_eff(zi, k, u)

# Realizar la integración utilizando la regla del trapecio
ec2 = np.trapz(ec1, k)

# Segunda integral
ec3 = np.trapz(ec2, u)

print("El resultado de la integral es:", ec3)

NameError: name 'np' is not defined